**Keystroke Dynamics**

**IMPORTACIOES**

In [17]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

**TRABAJO DE DATOS**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# importamos
df=pd.read_csv("/content/drive/MyDrive/practicas 2/keystrokes_dynamics/Datos.csv",sep=";")

Importante notar que nuestro conjunto de datos tiene 2 oraciones diferentes, ambas identificadas como modo: 1 y modo: 2 por lo que las diferenciamos para trabjar más ordenadamente.

In [4]:
# difereciamos los modos
dfM1,dfM2=df[df.modo==1],df[df.modo==2]

La columna tecla nos indica el código Ascii de la letra correspondiente al caracter de la oración, para que nos sea más simple entender nuestros datos creamos una nueva columna con el valor más legible, con la letra.

In [5]:
# Creamos una nueva columna "letra" compuesta por una lista que tiene cada codigo ascii bajo la función chr()
dfM1["letra"],dfM2["letra"]= [chr(x) for x in dfM1.tecla],[chr(x) for x in dfM2.tecla]

In [6]:
dfM1=dfM1.reset_index().drop("index",axis=1)
dfM2=dfM2.reset_index().drop("index",axis=1)

Nuestras variables descriptivas son Up y Down, ambas nos indican el momento en que una tecla fue apretada y soltada. A partir de ella podemos crear nuevas variables que nos den más información, entre ellas creamos:
- Down Actual menos Down Anterior ----- **(dAc-dAn)**
- Down Actual menos Up Anterior --------- **(dAc-uAn)**
- Up Actual menos Down Anterior --------- **(uAc-dAn)**
- Up Actual menos Up Anterior ------------- **(uAc-uAn)**
- Up Actual menos Down Actual ----------- **(uAc-dAc)**

In [7]:
# Creamos nuestras nuevas variables a partir de las diferencias, para ello utilizamos la función shift
dfM1["dAc-dAn"], dfM2["dAc-dAn"]=dfM1.down-dfM1.down.shift(1), dfM2.down-dfM2.down.shift(1)
dfM1["dAc-uAn"], dfM2["dAc-uAn"]=dfM1.down-dfM1.up.shift(1), dfM2.down-dfM2.up.shift(1)
dfM1["uAc-dAn"], dfM2["uAc-dAn"]=dfM1.up-dfM1.down.shift(1), dfM2.up-dfM2.down.shift(1)
dfM1["uAc-uAn"], dfM2["uAc-uAn"]=dfM1.up-dfM1.up.shift(1), dfM2.up-dfM2.up.shift(1)
dfM1["uAc-dAc"], dfM2["uAc-dAc"]=dfM1.up - dfM1.down, dfM2.up - dfM2.down

Aún así todavía tenemos un problema, al utilizar la función shift para realizar nuestros calculos, va a haber un momento en el que el primer down de cada práctica lo estemos comprando con un el último valor de la práctica anterior. Por ende ese valor no nos estaría aportando información si no más bien estaría distorsionando los datos actuales. Por lo que concluímos que en todos los comienzos de prácticas las variables descrpitivas van a ser cero ya que no se pueden comprar con ningún caso anterior.

In [8]:
# Reemplazamos en los momentos de down=0 por cero a todas las demás variables descriptivas
dfM1.loc[dfM1.down==0, "dAc-dAn"], dfM2.loc[dfM2.down==0, "dAc-dAn"] = 0, 0
dfM1.loc[dfM1.down==0, "dAc-uAn"], dfM2.loc[dfM2.down==0, "dAc-uAn"] = 0, 0
dfM1.loc[dfM1.down==0, "uAc-dAn"], dfM2.loc[dfM2.down==0, "uAc-dAn"] = 0, 0
dfM1.loc[dfM1.down==0, "uAc-uAn"], dfM2.loc[dfM2.down==0, "uAc-uAn"] = 0, 0

In [9]:
df_M1Clasico=dfM1
df_M1Clasico

,tiempo_id,usuario,practica,tecla,up,down,modo,letra,dAc-dAn,dAc-uAn,uAc-dAn,uAc-uAn,uAc-dAc
0,1,1,1,69,43,0,1,E,0.0,0.0,0.0,0.0,43
1,2,1,1,83,237,157,1,S,157.0,114.0,237.0,194.0,80
2,3,1,1,84,458,330,1,T,173.0,93.0,301.0,221.0,128
3,4,1,1,65,571,426,1,A,96.0,-32.0,241.0,113.0,145
4,5,1,1,32,650,572,1,,146.0,1.0,224.0,79.0,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81570,131667,70,9,65,54487,54391,1,A,219.0,147.0,315.0,243.0,96
81571,131668,70,9,82,54698,54631,1,R,240.0,144.0,307.0,211.0,67
81572,131669,70,9,73,54781,54722,1,I,91.0,24.0,150.0,83.0,59
81573,131670,70,9,79,54929,54868,1,O,146.0,87.0,207.0,148.0,61


**Qué es un bigrama?**
    Un bigrama es un conjunto de dos letras, sílabas o palabras. Deteniendonos a pensar, trabajar con bigramas nos dará muchas más información ya que los tiempos de nuestras variables pueden diferenciarse dependiendo el par con el que se las tecleo. Por ejemplo: Si nosotros escribimos la palabra "ola" la letra "l" viene después de una "o" y la distancia entre la "o" y la "l" es mínima, mientras que la palabra "altura" la "l" viene después de la "a" cuya distancia es considerable. Por lo que trabajar con bigramas va a ser mucho más preciso que con las letras individuales.

In [10]:
#Consturcción de la columna bigrama

# Creamos una nueva columna que contenga la letra anterior
dfM1["Letra_Anterior"], dfM2["Letra_Anterior"]= dfM1.groupby(["usuario","practica"])["letra"].shift(), dfM2.groupby(["usuario","practica"])["letra"].shift()

# Creamos nuestros bigramas uniendo la letra actual con su anterior
dfM1["bigrama"], dfM2["bigrama"]=dfM1.letra + dfM1.Letra_Anterior , dfM2.letra + dfM2.Letra_Anterior

Filtramos y ordenamos las columnas que vamos a necesitar

In [11]:
dfM1=dfM1[["usuario","bigrama",'practica',"dAc-dAn","dAc-uAn","uAc-dAn","uAc-uAn","uAc-dAc"]].reset_index().drop("index",axis=1)
dfM2=dfM2[["usuario","bigrama",'practica',"dAc-dAn","dAc-uAn","uAc-dAn","uAc-uAn","uAc-dAc"]].reset_index().drop("index",axis=1)
#display(dfM1)
#display(dfM2)

Ahora que ya tenemos nuestras variables listas para trabajar, debemos ver la forma en que vamos a ordenar nuestros datos. A nosotros nos interesaría que cada fila de nuestros datos correspondan a una práctica completa. De tal forma tendríamos cada bigrama de la oración en una única fila por lo que debemos transversar nuestros datos. ¡Importante!: Por cada bigrama vamos a tener 5 columnas más ya que tenemos 5 variables descriptivas por bigrama.

In [12]:
df_pivotM1=dfM1.pivot_table(values=["dAc-dAn","dAc-uAn","uAc-dAn","uAc-uAn","uAc-dAc"],
                           index=['usuario','practica'],
                           columns=['bigrama'])
df_pivotM1=df_pivotM1.reset_index()
#df_pivotM1

**MODELOS DE PREDICCIÓN**

**Lógica para nuestro mejor modelo posible**

Hemos sido capaz de crear 5 features, por lo que uno pensaría que usando las 5 no vamos a estar obviando información pero no siempre sucede esto. A veces vamos a tener columnas que no nos aporten información relevante, a simple vista paracen darnos nueva información pero en realidad nos están dando la misma información contada de distinta forma.

Por otro lado, a veces una combinación específica de variables nos va a estar diciendo mucho más que otra combinación. Ya que nosotros los seres humanos a ojo no nos podemos dar cuenta de esta relación vamos a probar nuestro modelo con todas las posibles combinaciones de variables posibles.

In [13]:
# Obtener todas las combinaciones posibles de nuestras variables

In [14]:
# Importamos herramienta
from itertools import combinations

In [15]:
# Creamos una lista con los nombres de nuestras variables
lista=["dAc-dAn","dAc-uAn","uAc-dAn","uAc-uAn","uAc-dAc"]
# Creamos una lista vacía que contendrá todas nuestras combinaciones
combinaciones_features= list()
# Iteramos por cada valor de nuestra lista
for n in range(len(lista)+1):
    # Agregamos la combinación n correspondiente
    combinaciones_features += list(combinations(lista,n))
#display(combinaciones_features)

In [55]:
combinaciones_features

[(),
 ('dAc-dAn',),
 ('dAc-uAn',),
 ('uAc-dAn',),
 ('uAc-uAn',),
 ('uAc-dAc',),
 ('dAc-dAn', 'dAc-uAn'),
 ('dAc-dAn', 'uAc-dAn'),
 ('dAc-dAn', 'uAc-uAn'),
 ('dAc-dAn', 'uAc-dAc'),
 ('dAc-uAn', 'uAc-dAn'),
 ('dAc-uAn', 'uAc-uAn'),
 ('dAc-uAn', 'uAc-dAc'),
 ('uAc-dAn', 'uAc-uAn'),
 ('uAc-dAn', 'uAc-dAc'),
 ('uAc-uAn', 'uAc-dAc'),
 ('dAc-dAn', 'dAc-uAn', 'uAc-dAn'),
 ('dAc-dAn', 'dAc-uAn', 'uAc-uAn'),
 ('dAc-dAn', 'dAc-uAn', 'uAc-dAc'),
 ('dAc-dAn', 'uAc-dAn', 'uAc-uAn'),
 ('dAc-dAn', 'uAc-dAn', 'uAc-dAc'),
 ('dAc-dAn', 'uAc-uAn', 'uAc-dAc'),
 ('dAc-uAn', 'uAc-dAn', 'uAc-uAn'),
 ('dAc-uAn', 'uAc-dAn', 'uAc-dAc'),
 ('dAc-uAn', 'uAc-uAn', 'uAc-dAc'),
 ('uAc-dAn', 'uAc-uAn', 'uAc-dAc'),
 ('dAc-dAn', 'dAc-uAn', 'uAc-dAn', 'uAc-uAn'),
 ('dAc-dAn', 'dAc-uAn', 'uAc-dAn', 'uAc-dAc'),
 ('dAc-dAn', 'dAc-uAn', 'uAc-uAn', 'uAc-dAc'),
 ('dAc-dAn', 'uAc-dAn', 'uAc-uAn', 'uAc-dAc'),
 ('dAc-uAn', 'uAc-dAn', 'uAc-uAn', 'uAc-dAc'),
 ('dAc-dAn', 'dAc-uAn', 'uAc-dAn', 'uAc-uAn', 'uAc-dAc')]

Para nuestros datos nosotros habíamos utilizado la función Pivot_table(), esta función nos permite agregar funciones extra a nustros datos, tales como el promedio, los maximos, minimos, desvíos, etc. Esa información sería de gran utilidad ya que podríamos comprar estos valores entre los distintos bigramas.

In [18]:
df_pivotM1=dfM1.pivot_table(values=["dAc-dAn","dAc-uAn","uAc-dAn","uAc-uAn","uAc-dAc"],
                           index=['usuario','practica'],
                           columns=['bigrama'],
                           aggfunc=[np.mean,np.max,np.min,np.std])
df_pivotM1=df_pivotM1.reset_index()
#display(df_pivotM1)

Teniendo en cuenta que cada función agregada puede o no sumar información, procederemos a ver todas las posibles combinaciones de las funciones.

In [19]:
# Creamos una lista con los nombres de nuestras funciones
lista=[np.mean,np.max,np.min,np.std]
# Creamos una lista vacía que contendrá todas nuestras combinaciones
combinaciones_func= list()
# Iteramos por cada valor de nuestra lista
for n in range(len(lista)+1):
    # Agregamos la combinación n correspondiente
    combinaciones_func += list(combinations(lista,n))
#display(combinaciones_func)

**Ahora si probemos los modelos a partir de todas las combinaciones posibles**

**Importamos**

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

**Modelo: kNN No estandarizado**

In [21]:
# Lógica de nuestro Modelo kNN

# Appendaremos a los mejores modelos en una lista para poder verlos en más detalle
lista_con_los_mejores_modelos=[]

# Iteramos por nuestra lista de las combinaciones de features
for i in range(1,len(combinaciones_features)):
    
    for func in range(1,len(combinaciones_func)):
        df_pivotM1=dfM1.pivot_table(values=combinaciones_features[i], # valores: las variables correspondientes a la comb i
                           index=['usuario','practica'],
                           columns=['bigrama'],
                            aggfunc=combinaciones_func[func]) # funciones: correspondientes a la comb func
    
        # Reseteamos el indice
        df_pivotM1= df_pivotM1.reset_index()

        # Dividimos nuestros datos descriptivos y los target
        X=df_pivotM1.drop(["usuario"],axis=1)
        y=df_pivotM1.usuario

        # Realizamos la división en Train y Test
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=123)

        # Aplicamos un filtro del score ya que son muchas combinaciones posibles
        score_filtro=0.80
    
        # Iteramos en este rango que será el valor de los vecinos de nuestro modelo kNN
        for x in range(1,10):
            
            # Creamos Modelo
            Modelo_knn = KNeighborsClassifier(n_neighbors=x,weights="distance",algorithm="auto",metric="manhattan")
            
            # Entreamos
            Modelo_knn.fit(X_train, y_train)
            
            # Lógica de filtrado y presentación de resultados
            
            #Accuracy de nuestro modelo
            Acc=Modelo_knn.score(X_test,y_test) 
            
            #Filtrado de los Accuracy de los distintos modelos
            if Acc>score_filtro:
                print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
                print("")
                print("Combinación df: " +str(combinaciones_features[i]))
                print("Combinacion funciones aplicadas: " + str(combinaciones_func[func]))
                print("========================================================")
                print("Vecinos: " + str(x))
                print("Score: " + str(Acc))
                score_filtro=Acc
            
            # Apendamos los Acc más altos con toda su información una lista
            if Acc > 0.97:
                lista_con_los_mejores_modelos.append([["Score", str(round(Acc,2))],["Features", str(combinaciones_features[i])],[("Vecinos", str(x))],["Funciones", str(combinaciones_func[func])]])
                

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Combinación df: ('dAc-dAn',)
Combinacion funciones aplicadas: (<function mean at 0x7fd331388cb0>,)
Vecinos: 1
Score: 0.8307692307692308
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Combinación df: ('dAc-dAn',)
Combinacion funciones aplicadas: (<function amin at 0x7fd331382cb0>,)
Vecinos: 1
Score: 0.8923076923076924
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Combinación df: ('dAc-dAn',)
Combinacion funciones aplicadas: (<function mean at 0x7fd331388cb0>, <function amin at 0x7fd331382cb0>)
Vecinos: 1
Score: 0.8615384615384616
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Combinación df: ('dAc-dAn',)
Combinacion funciones aplicadas: (<function amax at 0x7fd331382a70>, <function amin at 0x7fd331382cb0>)
Vecinos: 1
Score: 0.8307692307692308
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Combinación df: ('dAc-dAn',)
Combinacion funciones ap

In [22]:
for x in lista_con_los_mejores_modelos:
    if float(x[0][1])>0.98:
        print((x))

[['Score', '1.0'], ['Features', "('uAc-dAc',)"], [('Vecinos', '1')], ['Funciones', '(<function amax at 0x7fd331382a70>,)']]
[['Score', '1.0'], ['Features', "('uAc-dAc',)"], [('Vecinos', '2')], ['Funciones', '(<function amax at 0x7fd331382a70>,)']]
[['Score', '1.0'], ['Features', "('uAc-dAc',)"], [('Vecinos', '1')], ['Funciones', '(<function amax at 0x7fd331382a70>, <function std at 0x7fd331388ef0>)']]
[['Score', '1.0'], ['Features', "('uAc-dAc',)"], [('Vecinos', '2')], ['Funciones', '(<function amax at 0x7fd331382a70>, <function std at 0x7fd331388ef0>)']]


Como podemos ver, tenemos 4 tipos de modelos con diferentes combinaciones que alcanzaron un Score del 100 %. Aunque calramente la variable y la función que nos brindan este resultado siempre se repiten. 
- Función: **uAc-dAc**
- Función: **np.max**

Lo único que cambia en nuestro modelos es la cantidad de vecinos que simplemente varía entre 1 y 2 mientras que las funciones aplicadas varían en la combinación: (np.max) y (np.max, np.std).

**Reahacemos nuestro modelo con la mejor combinación** ('uAc-dAc') y (np.max)

In [23]:
# Usamos el Pivot_table con las métricas que mejor nos dieron previamente
df_testeo=dfM1.pivot_table(values='uAc-dAc', 
                           index=['usuario','practica'],
                           columns=['bigrama'],
                            aggfunc=np.max)
df_testeo=df_testeo.reset_index()

Ya que queremos comprobar que nuestro modelo realmente prediga bien , vamos a robarnos unas filas para testearlas luego y comprobarlo. Para ello creamos unos df con los valores y eliminamos esas filas de nuestro df_testeo.

In [24]:
# Me guardo para predecir 
df_testeo_pre1, df_testeo_pre2, df_testeo_pre3 = df_testeo[231:232], df_testeo[21:22], df_testeo[314:315]

In [25]:
# Las eliminamos del data frame ya que no queremos que nuestro modelo sepa de ellas

df_testeo.drop([231], axis=0, inplace=True)
df_testeo.drop([21], axis=0, inplace=True)
df_testeo.drop([314], axis=0, inplace=True)

**Separamos nuestras X e y**

In [26]:
X=df_testeo.drop(["usuario"],axis=1)
y=df_testeo.usuario

**Modelo**

In [27]:
Modelo_knn = KNeighborsClassifier(n_neighbors=1,weights="distance",algorithm="auto",metric="manhattan")
Modelo_knn.fit(X, y)

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')

Dividimos los datos que vamos a testear luego en sus X e y

In [28]:
# División de X e y de los valores a predecir
X_para_predict1, y_para_predict1= df_testeo_pre1.drop(["usuario"],axis=1), df_testeo_pre1.usuario
X_para_predict2, y_para_predict2= df_testeo_pre2.drop(["usuario"],axis=1), df_testeo_pre2.usuario
X_para_predict3, y_para_predict3= df_testeo_pre3.drop(["usuario"],axis=1), df_testeo_pre3.usuario

**Veamos las predicciones**

In [29]:
# Classes_ nos muestra el orden de los usuarios que toma scikit-learn, nos sirve para comprobar el orden del predict_proba
display("Orden de Usuarios: " + str(Modelo_knn.classes_))

# Predict_proba nos dice las probabilidades pronosticadas de nuestro modelo.
display("Probabilidades: " + str(Modelo_knn.predict_proba(X_para_predict1)))

# Predict nos dice la predicción
display("Predicción: " + str(Modelo_knn.predict(X_para_predict1)))

# El valor a predecir
display("El valor a predecir era: " + str(y_para_predict1.iloc[0]))

'Orden de Usuarios: [ 1 62 64 65 66 67 68 69 70 72 73 74 75 76 77 79]'

'Probabilidades: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]'

'Predicción: [72]'

'El valor a predecir era: 72'

In [30]:
display("Orden de Usuarios: " + str(Modelo_knn.classes_))
display("Probabilidades:   " + str(Modelo_knn.predict_proba(X_para_predict2)))
display("Predicción: " + str(Modelo_knn.predict(X_para_predict2)))
display("El valor a predecir era: " + str(y_para_predict2.iloc[0]))

'Orden de Usuarios: [ 1 62 64 65 66 67 68 69 70 72 73 74 75 76 77 79]'

'Probabilidades:   [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]'

'Predicción: [1]'

'El valor a predecir era: 1'

In [31]:
display("Orden de Usuarios: " + str(Modelo_knn.classes_))
display("Probabilidades:   " + str(Modelo_knn.predict_proba(X_para_predict3)))
display("Predicción: " + str(Modelo_knn.predict(X_para_predict3)))
display("El valor a predecir era: " + str(y_para_predict3.iloc[0]))

'Orden de Usuarios: [ 1 62 64 65 66 67 68 69 70 72 73 74 75 76 77 79]'

'Probabilidades:   [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]'

'Predicción: [76]'

'El valor a predecir era: 76'

**CONCLUSIÓN**:

Llegamos a la conclusión que con una única variable tal como la diferencia entre el momento que solte las teclas y el momento en que las solte, y el calculo de np.max sobre los bigramas, nuestro modelo predice de forma exacta. Lo cual es muy positivo pero puede que aún podamos mejorarlo mucho más. Preguntas a responder:
- ¿Acaso con menos prácticas nuestro modelo responde igual de bien?
- Si la oración fuera más corta, ¿predice igual de bien?

Con estas preguntas planteamos una segunda parte del experimento, **la optimización.**

**OPTIMIZACIÓN PRIMERA PARTE (PRACTICAS)**

En esta primer parte de la optimización vamos a plantear el mismo modelo aunque vamos a disminuir las prácticas de los usaurios.

In [32]:
# ¿Cuantas prácticas tiene cada usuario?
chequeo=dfM1.groupby(["usuario","practica"],as_index=False)["usuario"].size()
display(chequeo.groupby(["usuario"],as_index=False).size())

,usuario,size
0,1,24
1,62,12
2,64,38
3,65,11
4,66,14
5,67,41
6,68,13
7,69,63
8,70,9
9,72,12


**!!!Veamos los distintos scores en los train con diferentes cantidades de prácticas por usuario** (no tomaremos al usuario 77 y 79 porque tienen solo 1 y 2 prácticas respectivamente)

In [33]:
# X representará la cantidad de prácticas, será el valor que filtre nuestro data frame antes de hacer el modelo.
for x in range(3,11):
    # Nuestro datos filtrados por la cantidad de prácticas menor al valor de x en la iteración y sin los usuarios 77 y 79
    dfM1_opt=dfM1[(dfM1.practica<x) & (dfM1.usuario != 77) &  (dfM1.usuario != 79)]
    
    # Aplicamos el Pivot_table con los valores que nos dieron correctamente en la otra parte del trabajo
    df_testeo_op=dfM1_opt.pivot_table(values='uAc-dAc', 
                           index=['usuario','practica'],
                           columns=['bigrama'],
                            aggfunc=np.max)
    df_testeo_op=df_testeo_op.reset_index()
    
    # Separamos las X e y
    X, y=df_testeo_op.drop(["usuario"],axis=1) , df_testeo_op.usuario
    
    # Se paramos los Train y los Test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=123)
    
    # Modelo óptimo ya encontrado
    Modelo_knn_op = KNeighborsClassifier(n_neighbors=1,weights="distance",algorithm="auto",metric="manhattan")
    
    #Entrenamos
    Modelo_knn_op.fit(X_train, y_train)
    
    #Los scores con las distintas prácticas
    print("El Acc del modelo con " + str(x) + " prácticas es: " + str(round(Modelo_knn_op.score(X_test,y_test),2)))
    print("-------------------------------------------------------------------------------------------------")

El Acc del modelo con 3 prácticas es: 0.67
-------------------------------------------------------------------------------------------------
El Acc del modelo con 4 prácticas es: 1.0
-------------------------------------------------------------------------------------------------
El Acc del modelo con 5 prácticas es: 1.0
-------------------------------------------------------------------------------------------------
El Acc del modelo con 6 prácticas es: 1.0
-------------------------------------------------------------------------------------------------
El Acc del modelo con 7 prácticas es: 0.88
-------------------------------------------------------------------------------------------------
El Acc del modelo con 8 prácticas es: 1.0
-------------------------------------------------------------------------------------------------
El Acc del modelo con 9 prácticas es: 1.0
-------------------------------------------------------------------------------------------------
El Acc del modelo 

**Increible**: Con tan solo 4 prácticas nuestro modelo tuvo un Score del 100%

**TRABAJEMOS CON 4 PRÁCTICAS**

Teniendo en cuenta que para esta prueba vamos a robarnos una práctica por usuario vamos a tomar 5 prácticas. De tal forma nuestro modelo entrenará con solo 4 y luego lo testearemos con las otras que nos sacamos.

**Datos**

In [34]:
df_4_practicas=dfM1[(dfM1.practica<6) & (dfM1.usuario != 77) &  (dfM1.usuario != 79)]
#display(df_4_practicas)

In [35]:
df_4_practicas=df_4_practicas.pivot_table(values='uAc-dAc', 
                           index=['usuario','practica'],
                           columns=['bigrama'],
                            aggfunc=np.max)
df_4_practicas=df_4_practicas.reset_index()

Nos robamos variables para predecir

In [36]:
# Ordenamos por prácticas por ende todos los primeros valores corresponderán a la primer prácica de cada usuario
df_4_practicas_a_predecir= df_4_practicas.sort_values(by="practica").iloc[0:14]

Las eliminamos de nuestro data frame para que nuestro modelo no sepa nada de ellas

In [37]:
# Las eliminamos tras pasarles el indice 
df_4_practicas.drop(df_4_practicas.sort_values(by="practica").iloc[0:14].index, axis=0, inplace=True)

**X e y a predecir**

In [38]:
# Separamos en X e y los usuarios a predecir
X_para_predecir_4p=df_4_practicas_a_predecir.drop("usuario",axis=1)
y_para_predecir_4p=df_4_practicas_a_predecir.usuario

**Creamos nuestro modelo**

**Separamos nuestras X e y.** (no tienen las anteriormente quitadas)

In [39]:
X=df_4_practicas.drop(["usuario"],axis=1)
y=df_4_practicas.usuario

**Modelo**

In [40]:
Modelo_knn_4p = KNeighborsClassifier(n_neighbors=1,weights="distance",algorithm="auto",metric="manhattan")
Modelo_knn_4p.fit(X, y)

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')

**Predicciones**

In [41]:
for x in range(len(X_para_predecir_4p)):
    print("----------------------------------------")
    print("Predicho: " + str(Modelo_knn_4p.predict(X_para_predecir_4p.iloc[x:x+1])))
    print("Valor real: " + str(y_para_predecir_4p.iloc[x:x+1].iloc[0]))
    print("----------------------------------------")

----------------------------------------
Predicho: [1]
Valor real: 1
----------------------------------------
----------------------------------------
Predicho: [69]
Valor real: 69
----------------------------------------
----------------------------------------
Predicho: [67]
Valor real: 67
----------------------------------------
----------------------------------------
Predicho: [70]
Valor real: 70
----------------------------------------
----------------------------------------
Predicho: [66]
Valor real: 66
----------------------------------------
----------------------------------------
Predicho: [75]
Valor real: 75
----------------------------------------
----------------------------------------
Predicho: [68]
Valor real: 68
----------------------------------------
----------------------------------------
Predicho: [76]
Valor real: 76
----------------------------------------
----------------------------------------
Predicho: [72]
Valor real: 72
-----------------------------------

**!!!Genial** Con tan solo 4 prácticas nuestro modelo fue capaz de predecir con total exactitud a cada usuario

**Hagamos un prueba más para asegurarnos que las 4 prácticas son suficientes**

Para ello vamos a tomar 4 prácticas por cada usuario de nuestro conjunto de datos y con eso entrenaremos nuestro modelo. Luego usaremos ese modelo entrenado para predecir todo el restante.

In [42]:
chequeo=dfM1.groupby(["usuario","practica"],as_index=False)["usuario"].size()
display(chequeo.groupby(["usuario"],as_index=False).size())

,usuario,size
0,1,24
1,62,12
2,64,38
3,65,11
4,66,14
5,67,41
6,68,13
7,69,63
8,70,9
9,72,12


**Datos**

In [43]:
df_4_final=dfM1[(dfM1.usuario != 77) &  (dfM1.usuario != 79)]
df_4_practicas=df_4_final.pivot_table(values='uAc-dAc', 
                           index=['usuario','practica'],
                           columns=['bigrama'],
                            aggfunc=[np.max])
df_4_practicas=df_4_practicas.reset_index()

**Valores para entrenar**

In [44]:
df_entrenamiento=df_4_practicas[(df_4_practicas.practica<4)]

**Valores para testear modelo**

In [45]:
df_testeo=df_4_practicas[(df_4_practicas.practica>3)]

**Separamos valores para el testeo**

In [46]:
X_testeo=df_testeo.drop("usuario",axis=1)
y_testeo=df_testeo.usuario

**Separamos valores para el modelo**

In [47]:
X_modelo=df_entrenamiento.drop("usuario",axis=1)
y_modelo=df_entrenamiento.usuario

**CREAMOS EL MODELO**

In [48]:
Modelo_knn_4p_final = KNeighborsClassifier(n_neighbors=1,weights="distance",algorithm="auto",metric="manhattan")
Modelo_knn_4p_final.fit(X_modelo, y_modelo)

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')

In [49]:
Modelo_knn_4p_final.score(X_testeo,y_testeo)

0.7607142857142857

El score de mi modelo habiendo sido entrenado con tan solo 4 prácticas por usuario y habiendolo testeado con todo el restante de datos es de 0.75, bastante más bajo que lo anterior. ¿El problema está con todos los usuarios? Veamos:

**FRASE CORTA**

In [50]:
df_corto=pd.DataFrame()
for x in range(0,len(df_M1Clasico)):
    if df_M1Clasico.loc[x].down==0:
        df_corto=pd.concat([df_corto,df_M1Clasico.loc[x:x+17]])

In [51]:
#Consturcción de la columna bigrama

# Creamos una nueva columna que contenga la letra anterior
df_corto["Letra_Anterior"]= df_corto.groupby(["usuario","practica"])["letra"].shift()

# Creamos nuestros bigramas uniendo la letra actual con su anterior
df_corto["bigrama"]=df_corto.letra + df_corto.Letra_Anterior 

In [52]:
df_corto=df_corto[["usuario","bigrama",'practica',"dAc-dAn","dAc-uAn","uAc-dAn","uAc-uAn","uAc-dAc"]].reset_index().drop("index",axis=1)

In [53]:
df_testeo_corto=df_corto.pivot_table(values='uAc-dAc', 
                           index=['usuario','practica'],
                           columns=['bigrama'],
                            aggfunc=np.max)
df_testeo_corto=df_testeo_corto.reset_index()

In [54]:
X, y=df_testeo_corto.drop(["usuario"],axis=1) , df_testeo_corto.usuario
    
# Se paramos los Train y los Test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=123)

# Modelo óptimo ya encontrado
Modelo_knn_op = KNeighborsClassifier(n_neighbors=1,weights="distance",algorithm="auto",metric="manhattan")

#Entrenamos
Modelo_knn_op.fit(X_train, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')